In [2]:
import csv
import pandas as pd
from scipy import stats

In [3]:
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
pd.set_option('display.max_colwidth', None)

In [4]:
def rearrange_col(dataframe, col_to_move, reference_col, right=True):
    col_list = dataframe.columns.values.tolist()
    col_list2 = [x for x in col_list if x != col_to_move]
    reference_idx = col_list2.index(reference_col)
    if right==True:
        col_list3 = []
        for y in col_list2:
            col_list3.append(y)
            if y == reference_col:
                col_list3.append(col_to_move)
        return dataframe[col_list3]
    else:
        col_list3 = []
        for y in col_list2:
            col_list3.append(y)
            if y == reference_col:
                col_list3.insert(-2, col_to_move)
        return  dataframe[col_list3]

In [5]:
def get_key(dic, val): 
    for key, value in dic.items(): 
         if val == value: 
             return key 
  
    return "not found"

In [ ]:
raw = pd.read_excel('/Users/songheekim/Google Drive/Primary/Projects/VerbVector/verblist_v2.11_wb.xlsx')

In [ ]:
raw = raw.set_index('Lemma')

In [ ]:
#raw['GrandIndex'].tolist()

In [ ]:
pd.to_numeric([raw["GrandIndex"], raw["unerg_pb"], raw["unacc_pb"], raw["tr_pb"], raw["totalN_pb"], raw["intr_g"], raw["tr_g"], raw["totalN_g"]], errors='ignore')

In [ ]:
raw_dic = raw.to_dict(orient='index')

In [ ]:
# l = [15, 59, 1000, -1, 'ab']
# l2 = [i for i in l if isinstance(i, int)]
# print(l2)

In [ ]:
raw_dic['work']

In [ ]:
### get the category from
for verb, row in raw_dic.items():
    #print(verb)
    row['most_frequent_pb'] = ''
    row['most_frequent_g'] = ''
    
    keys_to_extract1 = ["unerg_pb", "unacc_pb", "tr_pb"]
    sub_dic1 = {key: row[key] for key in keys_to_extract1 if pd.notna(row[key])}
    #tup1 = list(sub_dic1.items())
    #print (tup1)
    if len(sub_dic1) > 0: 
        max1 = max(sub_dic1.values())
        pb_cat = get_key(sub_dic1, max1)
        row['most_frequent_pb'] = pb_cat[0:-3]
    
    keys_to_extract2 = ["intr_g", "tr_g"]
    sub_dic2 = {key: row[key] for key in keys_to_extract2 if pd.notna(row[key])}
    if len(sub_dic2) > 0: 
        max2 = max(sub_dic2.values())
        g_cat = get_key(sub_dic2, max2)
        row['most_frequent_g'] = g_cat[0:-2]
    
    #print (l)
    #l2 = [i for i in l if isinstance(i, float)]
#     l2 = [number for number in l if pd.notna(number)]
#     if len(l2) > 0: 
#         mx = max(l2)
#         pb_cat = get_key(mx)
#         row['most_frequent_pb'] = pb_cat[0:-3]
#     m = [row['intr_g'], row['tr_g']]
#     m2 = [number for number in m if pd.notna(number)]
#     if len(m2) > 0: 
#         mxx = max(m2)
#         g_cat = get_key(mxx)
#         row['most_frequent_g'] = g_cat[0:-2]

In [ ]:
### fill in "SynClassFinal" based on most_frequent_pb, most_frequent_g
for verb, row in raw_dic.items():
    #print (verb)
    l1 = [row['most_frequent_pb'], row['most_frequent_g']]
    l2 = [i for i in l1 if len(i)>0]
    #print (len(l2), l2)
    if len(l2)==0:
        row['SynClassCorpus']='none'
    elif len(l2)==1:
        row['SynClassCorpus']=l2[0]
    elif len(l2)==2:
        if l2[0]==l2[1]:
            row['SynClassCorpus']=l2[0]
        else:
            if l2[0]=='tr' or l2[1]=='tr':
                row['SynClassCorpus']='conflict'
            else:
                row['SynClassCorpus']=l2[0]
                
                
    else:
        print (verb, " needs check")
#     print (row['SynClassCorpus'])
#     print ('============')

In [ ]:
raw_upd = pd.DataFrame.from_dict(raw_dic, orient='index')

In [ ]:
# raw_upd.head()

In [ ]:
#raw_upd

In [ ]:
raw_upd = rearrange_col(raw_upd, 'most_frequent_g', 'totalN_g', right=True)
raw_upd = rearrange_col(raw_upd, 'most_frequent_pb', 'totalN_pb', right=True)

In [ ]:
raw_upd2 = raw_upd.sort_values(by=['SynClassCorpus', 'GrandIndex'])

In [ ]:
raw_upd2

In [ ]:
raw_upd2.to_excel('/Users/songheekim/Google Drive/Primary/Projects/VerbVector/verblist_v2.12.xlsx')

In [ ]:
unerg = raw_upd2.loc[raw_upd2['SynClassCorpus']=='unerg']
unacc = raw_upd2.loc[raw_upd2['SynClassCorpus']=='unacc']
trans = raw_upd2.loc[raw_upd2['SynClassCorpus']=='tr']
conflict = raw_upd2.loc[raw_upd2['SynClassCorpus']=='conflict']
nodata = raw_upd2.loc[raw_upd2['SynClassCorpus']=='none']

In [ ]:
raw_upd2.head()

In [ ]:
###########PART 2###############

In [6]:
raw2 = pd.read_excel('/Users/songheekim/Google Drive/Primary/Projects/VerbVector/verblist_v2.12_wb.xlsx')

In [7]:
###google syntactic N-gram
ngram_raw = pd.read_csv('/Users/songheekim/Documents/verb_transitivity/verb_transitivity.tsv', delimiter='\t', index_col='verb')

In [8]:
ngram_raw.head()

,intrans,trans,xtrans,ditrans,total,percent_intrans,percent_trans,percent_ditrans
verb,,,,,,,,
expect,744348.0,197175.0,67.0,463.0,942053.0,0.790134,0.209304,0.000491
expected,2342329.0,66846.0,23.0,764.0,2409962.0,0.971936,0.027737,0.000317
expecting,158713.0,84397.0,27.0,575.0,243712.0,0.651232,0.346298,0.002359
expectorate,2075.0,273.0,0.0,0.0,2348.0,0.883731,0.116269,0.000000
expectorated,4035.0,115.0,0.0,0.0,4150.0,0.972289,0.027711,0.000000


In [ ]:
ngram_raw.iloc[:,0]

In [ ]:
ngram_raw.shape

In [9]:
ngram_trimmed = ngram_raw[~ngram_raw.index.duplicated(keep='first')]

In [ ]:
ngram_trimmed.shape

In [ ]:
#ngram_trimmed.loc['expect']

In [ ]:
# keylist = ngram_raw.index.tolist()
# [(i, keylist.count(i)) for i in keylist if keylist.count(i)!=1]

In [11]:
ngram_dict = ngram_trimmed.to_dict(orient='index')

In [12]:
ngram_dict

{'expect': {'intrans': 744348.0,
  'trans': 197175.0,
  'xtrans': 67.0,
  'ditrans': 463.0,
  'total': 942053.0,
  'percent_intrans': 0.7901338884330287,
  'percent_trans': 0.2093035105243548,
  'percent_ditrans': 0.0004914797787385635},
 'expected': {'intrans': 2342329.0,
  'trans': 66846.0,
  'xtrans': 23.0,
  'ditrans': 764.0,
  'total': 2409962.0,
  'percent_intrans': 0.9719360720210526,
  'percent_trans': 0.027737366813252656,
  'percent_ditrans': 0.000317017446748123},
 'expecting': {'intrans': 158713.0,
  'trans': 84397.0,
  'xtrans': 27.0,
  'ditrans': 575.0,
  'total': 243712.0,
  'percent_intrans': 0.6512317817752101,
  'percent_trans': 0.3462980895483193,
  'percent_ditrans': 0.002359342174369748},
 'expectorate': {'intrans': 2075.0,
  'trans': 273.0,
  'xtrans': 0.0,
  'ditrans': 0.0,
  'total': 2348.0,
  'percent_intrans': 0.8837308347529813,
  'percent_trans': 0.11626916524701872,
  'percent_ditrans': 0.0},
 'expectorated': {'intrans': 4035.0,
  'trans': 115.0,
  'xtrans'

In [13]:
raw2 = raw2.set_index('Lemma')

In [15]:
raw2_dic = raw2.to_dict(orient='index')

In [16]:
raw2_dic

{'work': {'GrandIndex': 1,
  'Index': 1,
  'LogFreqHAL': 13.02,
  'Concreteness': 3.48,
  'AgeofAcqsn': 5.86,
  'Database': 'CREA',
  'Trans_Intrans': 'both',
  'SynClassLitrt': 'unergative',
  'SynClassMemo': 'unergative',
  'SynClassSK': 'unergative',
  'unerg_pb': 0.78,
  'unacc_pb': 0.044,
  'tr_pb': 0.16,
  'totalN_pb': 431.0,
  'most_frequent_pb': 'unerg',
  'intr_g': nan,
  'tr_g': nan,
  'totalN_g': nan,
  'most_frequent_g': nan,
  'Note': nan,
  'SynClassCorpus': 'unerg',
  'SynClassFinal': nan,
  'AspClass_Consensus': 'Process',
  'SemanticType_JB': 'other - general process',
  'SemanticType_JB_edit': 'other - general process',
  'Typicality_JB': 5.0,
  'SemanticType_LC': 'other',
  'Typicality_LC': nan,
  'SemClass_Consensus': 'other',
  'CONSENSUS_JB': 'other',
  'SemanticTypeLevin': 'verbs of creating and transformation',
  'SemanticTypeEntry': '26.5knead',
  'Percentage_dom_PoS': 0.57,
  'Length': 4,
  'Ortho_N': 9,
  'Phono_N': 25,
  'OLD': 1.5,
  'PLD': 1.2,
  'NPhon': 

In [18]:
exceptlist = [] ### find verbs that do not appear in Google n-gram
for key, value in raw2_dic.items(): ### update raw2_dic with Google-ngram percentages
    try: 
        raw2_dic[key]['intr_google'] = round(ngram_dict[key]['percent_intrans'],4)
        raw2_dic[key]['trans_google'] = round(ngram_dict[key]['percent_trans'],4)
    except: 
        raw2_dic[key]['intr_google'] = 'notfound'
        raw2_dic[key]['trans_google'] = 'notfound'
        exceptlist.append(key)

In [19]:
#raw2_dic['break']

{'GrandIndex': 92,
 'Index': 3,
 'LogFreqHAL': 10.774,
 'Concreteness': 3.71,
 'AgeofAcqsn': 5.3,
 'Database': 'CREA',
 'Trans_Intrans': 'both',
 'SynClassLitrt': 'unaccusative',
 'SynClassMemo': 'unaccusative',
 'SynClassSK': 'unaccusative',
 'unerg_pb': 0.1,
 'unacc_pb': 0.41,
 'tr_pb': 0.47,
 'totalN_pb': 135.0,
 'most_frequent_pb': 'tr',
 'intr_g': 0.18,
 'tr_g': 0.28,
 'totalN_g': 145.0,
 'most_frequent_g': 'tr',
 'Note': 'break in',
 'SynClassCorpus': 'tr',
 'SynClassFinal': 'transitive',
 'AspClass_Consensus': 'Achievement',
 'SemanticType_JB': 'ch of cohesiveness, obj dir action or transfer',
 'SemanticType_JB_edit': 'ch of cohesiveness, obj dir action or transfer',
 'Typicality_JB': 4.0,
 'SemanticType_LC': 'obj dir action/ch of cohesiveness',
 'Typicality_LC': 4.0,
 'SemClass_Consensus': 'obj-directed action',
 'CONSENSUS_JB': 'obj-directed action',
 'SemanticTypeLevin': 'verbs of removing, verbs of separating and disassembling, verbs of involving the body, verbs of change of

In [20]:
raw_upd3 = pd.DataFrame.from_dict(raw2_dic, orient='index') ### df with Google n-gram columns

In [21]:
raw_upd3 = rearrange_col(raw_upd3, 'intr_google', 'SynClassFinal', right=True)
raw_upd3 = rearrange_col(raw_upd3, 'trans_google', 'intr_google', right=True)

In [22]:
raw_upd3.head()

,GrandIndex,Index,LogFreqHAL,Concreteness,AgeofAcqsn,Database,Trans_Intrans,SynClassLitrt,SynClassMemo,SynClassSK,unerg_pb,unacc_pb,tr_pb,totalN_pb,most_frequent_pb,intr_g,tr_g,totalN_g,most_frequent_g,Note,SynClassCorpus,SynClassFinal,intr_google,trans_google,AspClass_Consensus,SemanticType_JB,SemanticType_JB_edit,Typicality_JB,SemanticType_LC,Typicality_LC,SemClass_Consensus,CONSENSUS_JB,SemanticTypeLevin,SemanticTypeEntry,Percentage_dom_PoS,Length,Ortho_N,Phono_N,OLD,PLD,NPhon,NSyll,I_Mean_RT,I_Mean_Accuracy,I_NMG_Mean_RT,I_NMG_Mean_Accuracy,N2_F,N3_F
work,1,1,13.020,3.48,5.86,CREA,both,unergative,unergative,unergative,0.780,0.0440,0.160,431.0,unerg,NaN,NaN,NaN,NaN,NaN,unerg,NaN,0.8878,0.1117,Process,other - general process,other - general process,5.0,other,NaN,other,other,verbs of creating and transformation,26.5knead,0.57,4,9,25,1.50,1.20,3,1,633.152,1.00,521.889,1.000,2421.10,1034.67
go,2,2,12.923,3.15,3.37,CREA,intransitive only,unergative,unergative PP arg,unergative,0.160,0.4300,0.020,1029.0,unacc,NaN,NaN,NaN,NaN,NaN,unacc,NaN,0.9567,0.0432,Achievement,ch of location,ch of location,5.0,ch of location,5.0,ch of location,ch of location,"verbs of existence, verbs of motion","47.7meander, 51.1advance",0.98,2,17,48,1.35,1.00,2,1,597.688,0.97,524.071,1.000,974.49,NaN
run,3,3,12.316,4.31,4.47,CREA,both,unergative,unergative,unergative,0.180,0.3400,0.440,374.0,tr,NaN,NaN,NaN,NaN,NaN,tr,NaN,0.7627,0.2365,Process,body mvt or posture,body movement or posture,5.0,manner of motion,5.0,manner of motion,manner of motion,"verbs of creating and transformation, verbs of existence, verbs of existence, verbs of motion","26.3prepare, 47.5.1swarm, 47.7meander, 51.3.2run",0.89,3,19,40,1.00,1.00,3,1,596.647,1.00,559.955,1.000,386.53,229.70
play,4,4,11.915,3.24,4.10,CREA,both,unergative,"unergative, transitive",unergative,0.097,0.1500,0.720,186.0,tr,0.33,0.57,150.0,tr,NaN,tr,transitive,0.5561,0.44,Process,social event,social event,5.0,social or interpersonal event,3.0,social or interpersonal event,social or interpersonal event,"verbs of creating and transformation, verbs of social interaction","26.7perform, 36.3meet",0.81,4,6,15,1.45,1.10,3,1,537.909,1.00,566.077,0.963,781.22,329.47
live,5,5,11.763,3.57,6.10,CREA,intransitive only,unergative/unaccusative,unergative PP arg,unergative,0.950,0.0078,0.047,129.0,unerg,NaN,NaN,NaN,NaN,NaN,unerg,NaN,0.9418,0.0581,State,existence,existence,5.0,existence,5.0,existence,existence,"verbs of ingesting, lodge verbs, verbs of existence","39.6gorge, 46lodge, 47.1exist",0.89,4,16,21,1.15,1.25,3,1,629.886,1.00,553.769,0.963,3645.64,606.44


In [29]:
### find out how well SynClassSK and Google Ngram match
unergSK = raw_upd3.loc[raw_upd3['SynClassSK']=='unergative']
unaccSK = raw_upd3.loc[raw_upd3['SynClassSK']=='unaccusative'] 
transSK = raw_upd3.loc[raw_upd3['SynClassSK']=='transitive'] 

In [49]:
unergBest = unergSK.loc[raw_upd3['intr_google']>raw_upd3['trans_google']]
unergFail = unergSK.loc[raw_upd3['intr_google']<raw_upd3['trans_google']]
print ('# of all unergative verbs is ', unergSK.shape[0])
print ('# of unergative verbs that are supported by Google N-gram is ', unergBest.shape[0])
print ('the unergative verbs that are not supported are: ', unergFail.index.tolist())

# of all unergative verbs is  89
# of unergative verbs that are supported by Google N-gram is  77
the unergative verbs that are not supported are:  ['leave', 'cross', 'shrug', 'inhale']


In [52]:
unaccBest = unaccSK.loc[raw_upd3['intr_google']>raw_upd3['trans_google']]
unaccFail = unaccSK.loc[raw_upd3['intr_google']<raw_upd3['trans_google']]
print ('# of all unaccusative verbs is ', unaccSK.shape[0])
print ('# of unaccusative verbs that are supported by Google N-gram is ', unaccBest.shape[0])
print ('the unaccusative verbs that are not supported are: ', unaccFail.index.tolist())

# of all unaccusative verbs is  177
# of unaccusative verbs that are supported by Google N-gram is  119
the unaccusative verbs that are not supported are:  ['open', 'break', 'change', 'resume', 'increase', 'improve', 'click', 'smell', 'stretch', 'gather', 'shake', 'integrate', 'multiply', 'smash', 'strengthen', 'shatter', 'accelerate', 'dissolve', 'spoil', 'lighten', 'tighten', 'sway', 'drown', 'loosen', 'diminish', 'weaken', 'straighten', 'lessen', 'revive', 'shorten', 'awaken', 'broaden', 'soften', 'enlarge', 'diffuse', 'consolidate', 'inflate', 'sharpen', 'fasten', 'intensify', 'lengthen', 'dampen', 'darken', 'deepen', 'vaporize', 'submerge']


In [53]:
transBest = transSK.loc[raw_upd3['intr_google']<raw_upd3['trans_google']]
transFail = transSK.loc[raw_upd3['intr_google']>raw_upd3['trans_google']]
print ('# of all transitive verbs is ', transSK.shape[0])
print ('# of transitive verbs that are supported by Google N-gram is ', transBest.shape[0])
print ('the transitive verbs that are not supported are: ', transFail.index.tolist())

# of all transitive verbs is  179
# of transitive verbs that are supported by Google N-gram is  148
the transitive verbs that are not supported are:  ['want', 'help', 'read', 'write', 'eat', 'survive', 'steal', 'mean', 'allow', 'expect', 'suggest', 'doubt', 'prefer', 'assume', 'realize', 'imagine', 'suppose', 'delete', 'spare', 'imply', 'extract', 'ruin', 'predict', 'observe', 'regret', 'assert', 'perceive', 'admire', 'forbid', 'ascertain', 'crave']


In [63]:
### update SynClassFinal based on Google Ngram
raw_upd3.SynClassFinal = '' ### clear the SynClassFinal column
raw_upd3.loc[(raw_upd3['SynClassSK']=='unergative') & (raw_upd3['intr_google']>raw_upd3['trans_google']), 'SynClassFinal']='unergative'
raw_upd3.loc[(raw_upd3['SynClassSK']=='unergative') & (raw_upd3['intr_google']<raw_upd3['trans_google']), 'SynClassFinal']='unergativefailed'
raw_upd3.loc[(raw_upd3['SynClassSK']=='unaccusative') & (raw_upd3['intr_google']>raw_upd3['trans_google']), 'SynClassFinal']='unaccusative'
raw_upd3.loc[(raw_upd3['SynClassSK']=='unaccusative') & (raw_upd3['intr_google']<raw_upd3['trans_google']), 'SynClassFinal']='unaccusativefailed'
raw_upd3.loc[(raw_upd3['SynClassSK']=='transitive') & (raw_upd3['intr_google']<raw_upd3['trans_google']), 'SynClassFinal']='transitive'

In [64]:
raw_upd3.head()

,GrandIndex,Index,LogFreqHAL,Concreteness,AgeofAcqsn,Database,Trans_Intrans,SynClassLitrt,SynClassMemo,SynClassSK,unerg_pb,unacc_pb,tr_pb,totalN_pb,most_frequent_pb,intr_g,tr_g,totalN_g,most_frequent_g,Note,SynClassCorpus,SynClassFinal,intr_google,trans_google,AspClass_Consensus,SemanticType_JB,SemanticType_JB_edit,Typicality_JB,SemanticType_LC,Typicality_LC,SemClass_Consensus,CONSENSUS_JB,SemanticTypeLevin,SemanticTypeEntry,Percentage_dom_PoS,Length,Ortho_N,Phono_N,OLD,PLD,NPhon,NSyll,I_Mean_RT,I_Mean_Accuracy,I_NMG_Mean_RT,I_NMG_Mean_Accuracy,N2_F,N3_F
work,1,1,13.020,3.48,5.86,CREA,both,unergative,unergative,unergative,0.780,0.0440,0.160,431.0,unerg,NaN,NaN,NaN,NaN,NaN,unerg,unergative,0.8878,0.1117,Process,other - general process,other - general process,5.0,other,NaN,other,other,verbs of creating and transformation,26.5knead,0.57,4,9,25,1.50,1.20,3,1,633.152,1.00,521.889,1.000,2421.10,1034.67
go,2,2,12.923,3.15,3.37,CREA,intransitive only,unergative,unergative PP arg,unergative,0.160,0.4300,0.020,1029.0,unacc,NaN,NaN,NaN,NaN,NaN,unacc,unergative,0.9567,0.0432,Achievement,ch of location,ch of location,5.0,ch of location,5.0,ch of location,ch of location,"verbs of existence, verbs of motion","47.7meander, 51.1advance",0.98,2,17,48,1.35,1.00,2,1,597.688,0.97,524.071,1.000,974.49,NaN
run,3,3,12.316,4.31,4.47,CREA,both,unergative,unergative,unergative,0.180,0.3400,0.440,374.0,tr,NaN,NaN,NaN,NaN,NaN,tr,unergative,0.7627,0.2365,Process,body mvt or posture,body movement or posture,5.0,manner of motion,5.0,manner of motion,manner of motion,"verbs of creating and transformation, verbs of existence, verbs of existence, verbs of motion","26.3prepare, 47.5.1swarm, 47.7meander, 51.3.2run",0.89,3,19,40,1.00,1.00,3,1,596.647,1.00,559.955,1.000,386.53,229.70
play,4,4,11.915,3.24,4.10,CREA,both,unergative,"unergative, transitive",unergative,0.097,0.1500,0.720,186.0,tr,0.33,0.57,150.0,tr,NaN,tr,unergative,0.5561,0.44,Process,social event,social event,5.0,social or interpersonal event,3.0,social or interpersonal event,social or interpersonal event,"verbs of creating and transformation, verbs of social interaction","26.7perform, 36.3meet",0.81,4,6,15,1.45,1.10,3,1,537.909,1.00,566.077,0.963,781.22,329.47
live,5,5,11.763,3.57,6.10,CREA,intransitive only,unergative/unaccusative,unergative PP arg,unergative,0.950,0.0078,0.047,129.0,unerg,NaN,NaN,NaN,NaN,NaN,unerg,unergative,0.9418,0.0581,State,existence,existence,5.0,existence,5.0,existence,existence,"verbs of ingesting, lodge verbs, verbs of existence","39.6gorge, 46lodge, 47.1exist",0.89,4,16,21,1.15,1.25,3,1,629.886,1.00,553.769,0.963,3645.64,606.44


In [66]:
colnames = list(raw_upd3.columns.values)
cols_to_test = ['LogFreqHAL', 'AgeofAcqsn', 'Percentage_dom_PoS', 'Length', 'Ortho_N', 'Phono_N',\
                'OLD', 'PLD', 'NPhon', 'NSyll', 'I_Mean_RT', 'I_Mean_Accuracy', 'I_NMG_Mean_RT', \
                'I_NMG_Mean_Accuracy', 'N2_F', 'N3_F']

In [70]:
for c in cols_to_test: 
    if raw_upd3.loc[pd.isna(raw_upd3[c])].shape[0] !=0: ### find any columns that contain NaN
        print (c)

N3_F


In [68]:
raw_upd3.loc[pd.isna(raw_upd3['N3_F'])] ## this returns the row where N3F is NaN

,GrandIndex,Index,LogFreqHAL,Concreteness,AgeofAcqsn,Database,Trans_Intrans,SynClassLitrt,SynClassMemo,SynClassSK,unerg_pb,unacc_pb,tr_pb,totalN_pb,most_frequent_pb,intr_g,tr_g,totalN_g,most_frequent_g,Note,SynClassCorpus,SynClassFinal,intr_google,trans_google,AspClass_Consensus,SemanticType_JB,SemanticType_JB_edit,Typicality_JB,SemanticType_LC,Typicality_LC,SemClass_Consensus,CONSENSUS_JB,SemanticTypeLevin,SemanticTypeEntry,Percentage_dom_PoS,Length,Ortho_N,Phono_N,OLD,PLD,NPhon,NSyll,I_Mean_RT,I_Mean_Accuracy,I_NMG_Mean_RT,I_NMG_Mean_Accuracy,N2_F,N3_F
go,2,2,12.923,3.15,3.37,CREA,intransitive only,unergative,unergative PP arg,unergative,0.16,0.43,0.02,1029.0,unacc,NaN,NaN,NaN,NaN,NaN,unacc,unergative,0.9567,0.0432,Achievement,ch of location,ch of location,5.0,ch of location,5.0,ch of location,ch of location,"verbs of existence, verbs of motion","47.7meander, 51.1advance",0.98,2,17,48,1.35,1.0,2,1,597.688,0.97,524.071,1.0,974.49,NaN


In [74]:
item_to_remove = raw_upd3.loc[pd.isna(raw_upd3['N3_F'])].index[0]
print (item_to_remove, ',', raw_upd3.loc[item_to_remove]['SynClassFinal'])

go , unergative


In [91]:
#### Create a dict that includes individual Series for each syntactic category (for each low-level property)
l = []
for c in cols_to_test:
    exec("%s={}" %c)
    
dict_list = [LogFreqHAL, AgeofAcqsn, Percentage_dom_PoS, Length, Ortho_N, Phono_N, OLD, PLD, NPhon, NSyll, \
             I_Mean_RT, I_Mean_Accuracy, I_NMG_Mean_RT, I_NMG_Mean_Accuracy, N2_F, N3_F]

dict_dict = dict(zip(cols_to_test, dict_list)) 

for k, v in dict_dict.items():
    print (k)
    for s in ['unergative', 'unaccusative', 'transitive']:
        v[s] = raw_upd3.loc[raw_upd3['SynClassFinal']==s][k].to_list()
#         print (v['unergative'])
#     print ('====')
#     print (v['unacc'])
#     print ('====')
#     print (v['tr'])
    print('unerg vs unacc:', stats.ttest_ind(v['unergative'], v['unaccusative']))
    print('unacc vs trans:', stats.ttest_ind(v['unaccusative'], v['transitive']))
    print('unerg vs trans:', stats.ttest_ind(v['unergative'], v['transitive']))
    print ('============================================================================================')
#     v['unerg-unacc'] = stats.ttest_ind(v['unerg'], v['unacc'])
#     v['unerg-tr'] = stats.ttest_ind(v['unerg'], v['tr'])
#     v['unacc-tr'] = stats.ttest_ind(v['unacc'], v['tr'])

LogFreqHAL
unerg vs unacc: Ttest_indResult(statistic=0.5770748873649814, pvalue=0.5645579166677265)
unacc vs trans: Ttest_indResult(statistic=-7.432345587990304, pvalue=1.4744963685504152e-12)
unerg vs trans: Ttest_indResult(statistic=-5.81284880703498, pvalue=2.1084233444637657e-08)
AgeofAcqsn
unerg vs unacc: Ttest_indResult(statistic=-3.439416911419329, pvalue=0.0007135823758899433)
unacc vs trans: Ttest_indResult(statistic=-1.318171668265498, pvalue=0.18858443264721164)
unerg vs trans: Ttest_indResult(statistic=-4.856304495764849, pvalue=2.2505428756713965e-06)
Percentage_dom_PoS
unerg vs unacc: Ttest_indResult(statistic=-3.393950173412773, pvalue=0.0008352483792223676)
unacc vs trans: Ttest_indResult(statistic=-2.7463982705386627, pvalue=0.00643822250191518)
unerg vs trans: Ttest_indResult(statistic=-6.299871828995127, pvalue=1.5695669865466674e-09)
Length
unerg vs unacc: Ttest_indResult(statistic=-2.1748387290012894, pvalue=0.030851997808501348)
unacc vs trans: Ttest_indResult(sta

In [92]:
raw_upd3.head()

,GrandIndex,Index,LogFreqHAL,Concreteness,AgeofAcqsn,Database,Trans_Intrans,SynClassLitrt,SynClassMemo,SynClassSK,unerg_pb,unacc_pb,tr_pb,totalN_pb,most_frequent_pb,intr_g,tr_g,totalN_g,most_frequent_g,Note,SynClassCorpus,SynClassFinal,intr_google,trans_google,AspClass_Consensus,SemanticType_JB,SemanticType_JB_edit,Typicality_JB,SemanticType_LC,Typicality_LC,SemClass_Consensus,CONSENSUS_JB,SemanticTypeLevin,SemanticTypeEntry,Percentage_dom_PoS,Length,Ortho_N,Phono_N,OLD,PLD,NPhon,NSyll,I_Mean_RT,I_Mean_Accuracy,I_NMG_Mean_RT,I_NMG_Mean_Accuracy,N2_F,N3_F
work,1,1,13.020,3.48,5.86,CREA,both,unergative,unergative,unergative,0.780,0.0440,0.160,431.0,unerg,NaN,NaN,NaN,NaN,NaN,unerg,unergative,0.8878,0.1117,Process,other - general process,other - general process,5.0,other,NaN,other,other,verbs of creating and transformation,26.5knead,0.57,4,9,25,1.50,1.20,3,1,633.152,1.00,521.889,1.000,2421.10,1034.67
go,2,2,12.923,3.15,3.37,CREA,intransitive only,unergative,unergative PP arg,unergative,0.160,0.4300,0.020,1029.0,unacc,NaN,NaN,NaN,NaN,NaN,unacc,unergative,0.9567,0.0432,Achievement,ch of location,ch of location,5.0,ch of location,5.0,ch of location,ch of location,"verbs of existence, verbs of motion","47.7meander, 51.1advance",0.98,2,17,48,1.35,1.00,2,1,597.688,0.97,524.071,1.000,974.49,NaN
run,3,3,12.316,4.31,4.47,CREA,both,unergative,unergative,unergative,0.180,0.3400,0.440,374.0,tr,NaN,NaN,NaN,NaN,NaN,tr,unergative,0.7627,0.2365,Process,body mvt or posture,body movement or posture,5.0,manner of motion,5.0,manner of motion,manner of motion,"verbs of creating and transformation, verbs of existence, verbs of existence, verbs of motion","26.3prepare, 47.5.1swarm, 47.7meander, 51.3.2run",0.89,3,19,40,1.00,1.00,3,1,596.647,1.00,559.955,1.000,386.53,229.70
play,4,4,11.915,3.24,4.10,CREA,both,unergative,"unergative, transitive",unergative,0.097,0.1500,0.720,186.0,tr,0.33,0.57,150.0,tr,NaN,tr,unergative,0.5561,0.44,Process,social event,social event,5.0,social or interpersonal event,3.0,social or interpersonal event,social or interpersonal event,"verbs of creating and transformation, verbs of social interaction","26.7perform, 36.3meet",0.81,4,6,15,1.45,1.10,3,1,537.909,1.00,566.077,0.963,781.22,329.47
live,5,5,11.763,3.57,6.10,CREA,intransitive only,unergative/unaccusative,unergative PP arg,unergative,0.950,0.0078,0.047,129.0,unerg,NaN,NaN,NaN,NaN,NaN,unerg,unergative,0.9418,0.0581,State,existence,existence,5.0,existence,5.0,existence,existence,"verbs of ingesting, lodge verbs, verbs of existence","39.6gorge, 46lodge, 47.1exist",0.89,4,16,21,1.15,1.25,3,1,629.886,1.00,553.769,0.963,3645.64,606.44


In [93]:
raw_upd3.to_excel('/Users/songheekim/Google Drive/Primary/Projects/VerbVector/verblist_v2.13.xlsx')

In [ ]:
### find the best verbs
# unergative = []
# unaccusative = []
# transitive = []
# failed_unergative=[]
# not_included=[]
# need_to_be_classified=[]

# for k, v in raw2_dic.items():
#     if v['SynClassSK'] == 'unergative' and v['SynClassSK'].startswith(v['SynClassFinal']):
#         unergative.append(k)
#     elif v['SynClassSK'] == 'unergative' and v['SynClassFinal']=='none':
#         unergative.append(k)
#     elif v['SynClassSK'] == 'unaccusative' and v['SynClassSK'].startswith(v['SynClassFinal']):
#         unaccusative.append(k)
#     elif v['SynClassSK'] == 'transitive' and v['SynClassSK'].startswith(v['SynClassFinal']):
#         transitive.append(k)
#     elif v['SynClassSK'] == 'unergative' and v['SynClassSK'].startswith(v['SynClassFinal'])==False:
#         failed_unergative.append(k)
#     else: 
#         not_included.append(k)
#         if v['Database']=='CREA':
#             need_to_be_classified.append(k)
#             print (k, 'is a verb in CREA and needs classficiation')

In [ ]:
# print('the # of unergatives: ', len(unergative))
# print('the # of unaccusative: ', len(unaccusative))
# print('the # of transitives: ', len(transitive))
# print('the # of failed unergatives: ', len(failed_unergative))
# print(len(need_to_be_classified), 'of them appear in CREA')

In [ ]:
# raw_upd3 = pd.DataFrame.from_dict(raw2_dic, orient='index')

In [ ]:
#raw_upd3.head()

In [ ]:
### label name update
# raw_upd3.loc[raw_upd3['SynClassFinal']=='unerg', 'SynClassFinal']='unergative'
# raw_upd3.loc[raw_upd3['SynClassFinal']=='unacc', 'SynClassFinal']='unaccusative'
# raw_upd3.loc[raw_upd3['SynClassFinal']=='tr', 'SynClassFinal']='transitive'

In [ ]:
# unergative_list = raw_upd3[raw_upd3['SynClassFinal']=='unergative'].index
# unaccusative_list = raw_upd3[raw_upd3['SynClassFinal']=='unaccusative'].index
# transitive_list = raw_upd3[raw_upd3['SynClassFinal']=='transitive'].index

In [ ]:
# print (unergative_list, "Unergative N=", len(unergative_list))
# print (unaccusative_list, "Unaccusative N=", len(unaccusative_list))
# print (transitive_list, "Transitive N=", len(transitive_list))

In [ ]:
#### find verbs that have NaN in low-level qualities
# for c in cols_to_test: 
#     if raw_upd3.loc[pd.isna(raw_upd3[c])].shape[0] !=0: ### find any columns that contain NaN
#         verb_to_remove = raw_upd3.loc[pd.isna(raw_upd3[c])].index[0]
#         print ("verb", "\'", verb_to_remove, "\'", "has NaN in", c)
        
#### drop that verb from a list (if one of unergative, unaccsuative, transitive)
# categories = [unergative, unaccusative, transitive]
# for i in range(len(categories)):
#     if verb_to_remove in categories[i]:
#         print (i, verb_to_remove)
#     else:
#         print (verb_to_remove, 'is not found')

### make sure that low-level quality columns only have a numeral
# for col in cols_to_test:
#     col_to_list = raw_upd3[col].tolist()
#     types = list(set([type(i) for i in col_to_list]))
#     print (col, types)

#### Everything other than Concreteness looks fine

#pd.to_numeric(raw_upd3["Concreteness"])

In [ ]:
# for k, v in raw2_dic.items():
#     if pd.isna(v['SynClassFinal']):
#         #print(k)
#         v['SynClassFinal'] = v['SynClassCorpus']

In [ ]:
# verb_w_2syncat = raw_upd[raw_upd['most_frequent_pb'].notnull() & raw_upd['most_frequent_g'].notnull()]

# verb_w_1syncat = raw_upd[raw_upd['most_frequent_pb'].notnull() | raw_upd['most_frequent_g'].notnull()]

# verb_wo_syncat = raw_upd[raw_upd['most_frequent_pb'].isnull() & raw_upd['most_frequent_g'].isnull()]

# verb_w_2syncat.shape #number of verbs that appear both in PB and G

# verb_wo_syncat.shape #number of verbs that do not appear in either

# verb_w_1syncat.shape #number of verbs that appear in PB or G

# verb_w_1syncat.shape[0] + verb_w_2syncat.shape[0] + verb_wo_syncat.shape[0]

# raw_upd.shape